In [1]:
from pgreaper import Table, copy_table
from collections import defaultdict
import re

## Read First Ten Lines

In [2]:
with open('ibtracs_v03r09_serial-'
          'name-mapping.txt', mode='r') as infile:
    for i, j in enumerate(infile):
        if i == 10:
            break
        print(j)

1842298N11080  NOT NAMED(td9636)

1845336N10074  NOT NAMED(td9636)

1848011S09079  NOT NAMED(td9636)

1848011S09080  XXXX848003(reunion)

1848011S15057  XXXX848002(reunion)

1848011S16057  NOT NAMED(td9636)

1848061S12075  XXXX848001(reunion) - NOT NAMED(td9636)

1848062S13069  NOT NAMED(td9636)

1848112S07084  XXXX848004(reunion)

1848112S07444  NOT NAMED(td9636)



In [3]:
with open('ibtracs_v03r09_serial-'
          'name-mapping.txt', mode='r') as infile:
    for i, j in enumerate(infile):
        if i == 50:
            break
            
        k = j.split('  ')
        print(k[0], k[1].split(' - '))

1842298N11080 ['NOT NAMED(td9636)\n']
1845336N10074 ['NOT NAMED(td9636)\n']
1848011S09079 ['NOT NAMED(td9636)\n']
1848011S09080 ['XXXX848003(reunion)\n']
1848011S15057 ['XXXX848002(reunion)\n']
1848011S16057 ['NOT NAMED(td9636)\n']
1848061S12075 ['XXXX848001(reunion)', 'NOT NAMED(td9636)\n']
1848062S13069 ['NOT NAMED(td9636)\n']
1848112S07084 ['XXXX848004(reunion)\n']
1848112S07444 ['NOT NAMED(td9636)\n']
1851080S15062 ['NOT NAMED(td9636)\n']
1851080S15063 ['XXXX851006(reunion)\n']
1851080S21060 ['XXXX851005(reunion)\n']
1851080S21420 ['NOT NAMED(td9636)\n']
1851175N26270 ['UNNAMED(hurdat_atl)', 'bal011851(atcf)', 'NOT NAMED(mlc_natl)\n']
1851181N19275 ['NOT NAMED(mlc_natl)\n']
1851187N22262 ['UNNAMED(hurdat_atl)', 'bal021851(atcf)\n']
1851192N12300 ['UNNAMED(hurdat_atl)', 'bal031851(atcf)\n']
1851214N14321 ['NOT NAMED(mlc_natl)\n']
1851228N13313 ['UNNAMED(hurdat_atl)', 'bal041851(atcf)', 'NOT NAMED(mlc_natl)\n']
1851241N27280 ['NOT NAMED(mlc_natl)\n']
1851250N20278 ['NOT NAMED(mlc_nat

In [4]:
names = Table(name='ibtracs_hurricane_names',
              col_names=['serial_no', 'names'])

with open('ibtracs_v03r09_serial-'
          'name-mapping.txt', mode='r') as infile:
    for i in infile:
        j = i.split('  ')
        names.append([j[0],
                     [k.replace('\n', '') for k in j[1].split(' - ')]])

In [5]:
names.p_key = 'serial_no'
names

,serial_no,names
,text primary key,jsonb
[0],'1842298N11080',['NOT NAMED(td9636)']
[1],'1845336N10074',['NOT NAMED(td9636)']
[2],'1848011S09079',['NOT NAMED(td9636)']
[3],'1848011S09080',['XXXX848003(reunion)']
[4],'1848011S15057',['XXXX848002(reunion)']
[5],'1848011S16057',['NOT NAMED(td9636)']
[6],'1848061S12075',"['XXXX848001(reunion)', 'NOT NAMED(td9636)']"
[7],'1848062S13069',['NOT NAMED(td9636)']
[8],'1848112S07084',['XXXX848004(reunion)']


## Create a Mapping of Hurricane Names to Organizations that Named Them

In [6]:
temp = names['names'][19]
temp

['UNNAMED(hurdat_atl)', 'bal041851(atcf)', 'NOT NAMED(mlc_natl)']

In [7]:
def split_name(name):
    # Split the hurricane name from the organization that named it
    reg = re.match('(.*)\((.*)\)', name)
    return (reg.group(1), reg.group(2))

In [8]:
temp1 = [split_name(i) for i in temp]
temp1

[('UNNAMED', 'hurdat_atl'), ('bal041851', 'atcf'), ('NOT NAMED', 'mlc_natl')]

In [9]:
temp2 = defaultdict(list)
for x, y in temp1:
    temp2[x].append(y)
temp2

defaultdict(list,
            {'NOT NAMED': ['mlc_natl'],
             'UNNAMED': ['hurdat_atl'],
             'bal041851': ['atcf']})

In [10]:
def map_names(names):
    kv = [split_name(i) for i in names]
    d = defaultdict(list)
    for x, y in kv:
        d[x].append(y)
    return d

In [11]:
names.apply('names', map_names)

In [12]:
names

,serial_no,names
,text primary key,jsonb
[0],'1842298N11080',"defaultdict(<class 'list'>, {'NOT NAMED': ['td9636']})"
[1],'1845336N10074',"defaultdict(<class 'list'>, {'NOT NAMED': ['td9636']})"
[2],'1848011S09079',"defaultdict(<class 'list'>, {'NOT NAMED': ['td9636']})"
[3],'1848011S09080',"defaultdict(<class 'list'>, {'XXXX848003': ['reunion']})"
[4],'1848011S15057',"defaultdict(<class 'list'>, {'XXXX848002': ['reunion']})"
[5],'1848011S16057',"defaultdict(<class 'list'>, {'NOT NAMED': ['td9636']})"
[6],'1848061S12075',"defaultdict(<class 'list'>, {'XXXX848001': ['reunion'], 'NOT NAMED': ['td9636']})"
[7],'1848062S13069',"defaultdict(<class 'list'>, {'NOT NAMED': ['td9636']})"
[8],'1848112S07084',"defaultdict(<class 'list'>, {'XXXX848004': ['reunion']})"


In [14]:
copy_table(names, dbname='us_wth')